In [1]:
#Model card: https://huggingface.co/databricks/dolly-v2-3b

In [6]:
%pip install "accelerate>=0.16.0,<1" "transformers[torch]>=4.28.1,<5" "torch>=1.13.1,<2"


Defaulting to user installation because normal site-packages is not writeable
  Using cached accelerate-0.23.0-py3-none-any.whl (258 kB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement torch<2,>=1.13.1 (from versions: 2.0.0, 2.0.1, 2.1.0)
ERROR: No matching distribution found for torch<2,>=1.13.1

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import torch
from transformers import pipeline

generate_text = pipeline(model="databricks/dolly-v2-3b", torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto")


ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install accelerate`

In [ ]:
baseline_sentence = 'can you drink alcoholic drink in populace in kingdom of denmark'
res = generate_text(baseline_sentence)

for row in ca_array:
  sentence = ''
  sentence = ' '.join(row)
  res = generate_text(sentence)

#largely looking for a boolean output


#res = generate_text("Explain to me the difference between nuclear fission and fusion.")
#print(res[0]["generated_text"])

Nuclear fission creates a new atom by fusing two smaller ones together. This releases a lot of energy in the form of nuclear radiation. The resulting nucleus creates smaller nuclei, which leads to infinite nuclear chain reaction.

Nuclear fusion, on the other hand, creates a new atom by combining atoms of hydrogen or helium together. This releases less energy than nuclear fission, but yields much brighter light. Sunlight is mostly composed of nuclear fusion. Nuclear fusion releases energy in the form of light.
